In [ ]:
# Helpful Encoding :https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin

#!pip install category_encoders
#!pip install xgboost 
import xgboost as xgb

In [2]:
#df = pd.read_csv("../input/train_features.csv", header=0)

df = pd.read_csv("train_features.csv", header=0)
df_test = pd.read_csv("test_features.csv",header=0)
df_labels = pd.read_csv("train_labels.csv", header=0)
df_labels['status_group'].value_counts()
df = df.merge(df_labels, on='id')

In [15]:
df.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [11]:
df_slim = df[df.status_group != 'functional needs repair']

In [14]:
df_slim.status_group.value_counts()

functional        32259
non functional    22824
Name: status_group, dtype: int64

## REVISIT THIS DATA. DROP NAN ROWS? 5%

In [17]:
# Based on the most voted answer we can easily define a function that gives us a dataframe to preview the missing values and the % of missing values in each column:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [20]:
missing_values_table(df)

Your selected dataframe has 41 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
scheme_name,28166,47.4
scheme_management,3877,6.5
installer,3655,6.2
funder,3635,6.1
public_meeting,3334,5.6
permit,3056,5.1
subvillage,371,0.6


In [21]:
counts_ = []
features = list(df.columns)

fc = []
for x in features: fc.append([x, df[x].nunique()])
pd.DataFrame(data=fc, columns = ['features','counts']).sort_values(by=['counts']).set_index(['features'])

,counts
features,
recorded_by,1
public_meeting,2
permit,2
status_group,3
source_class,3
quantity_group,5
quantity,5
management_group,5
quality_group,6


In [41]:
# Encode my Y label and return a list of my labels. 
def labeler(dataframe, column):
    from sklearn.preprocessing import LabelEncoder
    dataframe =  dataframe.copy()
    le = LabelEncoder()
    dataframe['labels'] = le.fit_transform(dataframe[column])
    labels = [0,1,2]
    label_names = list(le.inverse_transform(labels))
    label_list = [labels, label_names]
    return dataframe['labels'], label_list


def slimlabeler(dataframe, column):
    from sklearn.preprocessing import LabelEncoder
    dataframe =  dataframe.copy()
    le = LabelEncoder()
    dataframe['labels'] = le.fit_transform(dataframe[column])
    labels = [0,1]
    label_names = list(le.inverse_transform(labels))
    label_list = [labels, label_names]
    return dataframe['labels'], label_list


# Turn Lat/Long into x,y,z, coord plane. 
def lat_long(dataframe):
    from math import cos, sin 
    dataframe =  dataframe.copy()
    dataframe['x_coord'] = dataframe.latitude.apply(lambda x: cos(x)) * dataframe.longitude.apply(lambda x: cos(x))
    dataframe['y_coord'] = dataframe.latitude.apply(lambda x: cos(x)) * dataframe.longitude.apply(lambda x: sin(x))
    dataframe['z_coord'] = dataframe.latitude.apply(lambda x: sin(x))
    dataframe = dataframe.drop(columns=['latitude', 'longitude'])
    return dataframe

# Fix silly boolean issue. 
def no_bool(dataframe, columns):
    dataframe =  dataframe.copy()
    for column in columns:
        dataframe[column] = dataframe[column].replace({True: 'Yes', False: 'No'})
    return dataframe
        

In [23]:
#for i in names:
#    temp_df.loc[temp_df[i].value_counts()[temp_df[i]].values < 20, i] = "RARE_VALUE"

In [25]:
# Now to decide how to handle the following madness:
"""
wpt_name	37400: These have some sites that encompass more than one well. 
date_recorded	356: DROP, missing data. 

# I'll deal with these ones after getting a new baseline. 
lga	125
funder	1897 - Trim?
    print(data.funder.value_counts()[data.funder.value_counts()>777])
    print(len(list(data.funder.value_counts()[data.funder.value_counts()<777])))
ward	2092 
installer	2145
scheme_name	2696 drop
subvillage	19287
""";

In [ ]:
df.waterpoint_type.value_counts()

In [55]:
df.region.value_counts()

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Name: region, dtype: int64

In [ ]:
df.ward.value_counts()

In [ ]:
## Feature	value_count()

# DROP status_group	3 : 
# DROP id	59400: because duh. 
# DROP recorded_by	1 : Only one source. 

# KEEP public_meeting	2 : Boolean to object. 
# KEEP permit	2: Boolean to object.  

# KEEP source_class	3 
# KEEP source	10: These two are sufficiently different. 
# DROP source_type	7 : This one is source but with less detail. Not ordered. 

# DROP quantity_group	5 : These two are exactly the same. Drop either. 
# KEEP  quantity	5 : enough, insufficient, dry, seasonal, unknown  - possibly cat/ordinal

# KEEP management	12 : Most detailed, not ordered. 
# DROP management_group	5 : less-detailed, summary-grouping. Not ordered.
# DROP scheme_management	12 : Missing Values, similar info to management. 

# DROP payment	7
# KEEP payment_type	7 : Shorter Column headers. Identical otherwise.

# KEEP water_quality	8
# DROP quality_group	6

# DROP waterpoint_type_group	6
# KEEP waterpoint_type	7

# DROP extraction_type_class	7
# DROP extraction_type_group	13
# KEEP extraction_type	18


### GEOGRAPHY 
# DROP longitude	57516: Replace with X,Y,Z coordinates
# DROP latitude	57517

# KEEP basin	9 - Non-Regional boarder may overlap with some other indicators of location. 

# DROP region	21 
# KEEP region_code	27: Region Code contains more detail than Region. 

# DROP district_code	20
# KEEP lga	125 : Lga is district with urban/rural flag added in on some values. Lga is most detailed.

# ward	2092
# gps_height	2428
# subvillage	19287
# wpt_name	37400


### OTHER
# construction_year	55
# num_private	65
# amount_tsh	98
# date_recorded	356
# population	1049


### High Cardinality
# funder	1897
# installer	2145
# PDROP  scheme_name	2696 - High cardinality, low 



In [150]:
# Fix or Enhance Features
data = df_slim
data = lat_long(data)
data = no_bool(data, ['permit', 'public_meeting'])

# Define those datasets
X = data
y, label_list = slimlabeler(df_slim, 'status_group')
y.head()

# feature thoughts
nope = ['recorded_by','construction_year', 'payment_type', 'quantity_group','source_type']
good = ['permit']
first = ['extraction_type','management', 'payment','water_quality', 'quantity','source', 'source_class','waterpoint_type']
second = ['extraction_type_group','management_group','quality_group','waterpoint_type_group']
third = ['extraction_type_class']

# Passthrough features. These are any I don't want to mess with. 
passthrough_features = []

#Testing
to_be_tested = ['subvillage']
test_remove = second + third
test_add_oh = []
test_add_binary = []
test_add_numeric = []

# Ones that aren't actually numeric.
binary_features = ['region_code','installer','scheme_name','date_recorded','lga','subvillage']
hashing_features = []

# Dropping features
drop_features_tune = ['id', 'status_group','recorded_by',"region",'funder','district_code',]
drop_features_testing = ['wpt_name']
drop_features = drop_features_tune + test_remove + drop_features_testing

# my need to convert region_code to 

# Ones to binary encode (high cardinality)
#binary_features = ['funder','installer','scheme_name', 'subvillage','region_code', 'district_code']

# Defining my one-hot variables. 
one_hot_features = list(X.select_dtypes(include=['object']))
numeric_features = list(X.select_dtypes(include=['float64', 'int64']))

# Remove features from duplicates if present. 
for x in drop_features:
    if x in one_hot_features: one_hot_features.remove(x)
    if x in numeric_features: numeric_features.remove(x)
    if x in binary_features: binary_features.remove(x)
    if x in hashing_features: hashing_features.remove(x)

for x in binary_features:
    if x in one_hot_features: one_hot_features.remove(x)
    if x in numeric_features: numeric_features.remove(x)
        
for x in hashing_features: 
    if x in one_hot_features: one_hot_features.remove(x)
    if x in numeric_features: numeric_features.remove(x)

binary_features = binary_features + one_hot_features
# List features
the_selected_features = passthrough_features + one_hot_features + numeric_features + binary_features + hashing_features

In [151]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, RobustScaler, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import f_classif, chi2, SelectKBest
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from category_encoders.hashing import HashingEncoder
from category_encoders.binary import BinaryEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

# Preprocessing pipelines for both numeric and categorical data.
# Using column_transformer https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html


# Define my custom pipeline functions for each type of data. Columns not expressly included are dropped.  


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

polynom_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()), 
    ('polynom', PolynomialFeatures())])

one_hot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', OrdinalEncoder()),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('binary', BinaryEncoder(drop_invariant=True,return_df=True))])

hashing_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('hashing', HashingEncoder())])

# Create preprocessor pipeline
PreProcessor = ColumnTransformer(
    transformers=[
#        ('pass', 'passthrough', passthrough_features),
        ('drop', 'drop', drop_features),
        #('hash', hashing_transformer, hashing_features),
        ('biy', binary_transformer, binary_features),
        ('num', numeric_transformer, numeric_features)
        #('o-h', one_hot_transformer, one_hot_features)
    ],
    n_jobs = -2)

# Lets test it.

In [152]:
# Test Train Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
print(X_train.shape, y_train.shape)
PreProcessor.fit_transform(X_train, y_train).shape

(49574, 42) (49574,)


(49574, 124)

## Feature Baseline

In [153]:
clf = make_pipeline(PreProcessor, LogisticRegression(max_iter=1000))

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_train, y_train))
print("model score: %.3f" % clf.score(X_val, y_val))

/home/britton/.conda/envs/basicNLP/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.775
model score: 0.782


## After Baseline

In [154]:
clf2 = make_pipeline(PreProcessor, XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.02,  
                      colsample_bytree = 0.3,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=100, 
                      reg_alpha = 0.4,
                      max_depth=17, 
                      gamma=1))

clf2.fit(X_train, y_train)
print("model score: %.3f" % clf2.score(X_train, y_train))
print("model score: %.3f" % clf2.score(X_val, y_val))

[23:50:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2198 extra nodes, 436 pruned nodes, max_depth=17
[23:50:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2950 extra nodes, 588 pruned nodes, max_depth=17
[23:50:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2654 extra nodes, 600 pruned nodes, max_depth=17
[23:50:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2922 extra nodes, 578 pruned nodes, max_depth=17
[23:50:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2710 extra nodes, 618 pruned nodes, max_depth=17
[23:50:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3172 extra nodes, 700 pruned nodes, max_depth=17
[23:50:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2982 extra nodes, 614 pruned nodes, max_depth=17
[23:50:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3392 extra nodes, 796 pru

KeyboardInterrupt: 

[23:50:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2652 extra nodes, 842 pruned nodes, max_depth=17
[23:50:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2624 extra nodes, 680 pruned nodes, max_depth=17
[23:50:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2634 extra nodes, 844 pruned nodes, max_depth=17
[23:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3560 extra nodes, 1406 pruned nodes, max_depth=17
[23:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2640 extra nodes, 752 pruned nodes, max_depth=17
[23:50:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3110 extra nodes, 958 pruned nodes, max_depth=17
[23:50:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2686 extra nodes, 774 pruned nodes, max_depth=17
[23:50:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3026 extra nodes, 982 pr

[23:50:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2350 extra nodes, 1164 pruned nodes, max_depth=17
[23:50:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2514 extra nodes, 1310 pruned nodes, max_depth=17
[23:50:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2424 extra nodes, 1588 pruned nodes, max_depth=17
[23:50:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2882 extra nodes, 1710 pruned nodes, max_depth=17
[23:50:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2494 extra nodes, 1824 pruned nodes, max_depth=17
[23:50:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2228 extra nodes, 2324 pruned nodes, max_depth=17
[23:50:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2244 extra nodes, 1734 pruned nodes, max_depth=17
[23:50:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2372 extra nodes, 

In [ ]:
# XGBoost
XGPipe = make_pipeline(PreProcessor, XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.3,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=100, 
                      reg_alpha = 0.4,
                      max_depth=17, 
                      gamma=1))

param_grid = {
    'xgbclassifier__max_depth':[5,10,15,17,19],
    'xgbclassifier__n_estimators':[50, 150, 250, 350, 450],
    'xgbclassifier__colsample_bytree':[.3, .8, .9],
    'xgbclassifier__learning_rate':[0.001, 0.0045, 0.0065, 0.010], 
    'xgbclassifier__reg_lambda':[0, 0.01, 0.10, 0.50, 1]
    }
grid = {}


# Fit on the train set, with grid search cross-validation
XGsearch = GridSearchCV(XGPipe, 
                        param_grid=param_grid, 
                        cv=3, 
                        scoring='accuracy',
                        verbose=10, n_jobs=-2)

XGsearch.fit(X_train, y_train)
print("train model score: %.3f" % XGsearch.score(X_train, y_train))
print("validate model score: %.3f" % XGsearch.score(X_val, y_val))
best_params = XGsearch.best_params_
print(best_params)


#scores = cross_validate(clf, X, y, cv=3, n_jobs=-2, scoring='accuracy',return_train_score=True, return_estimator=True)
#print(scores)
#print('Accuracy: %.3f stdev: %.2f' % (np.mean(np.abs(scores)), np.std(scores)))
#clf.fit(X_train,y_train)

# You can just drop it into a pandas dataframe and BOOM: pretty print! 
# pd.DataFrame(scores).rename(columns={"test_score": 'validation_score'})
# The test score is actually the scores from each validation cycle.

Fitting 3 folds for each of 1500 candidates, totalling 4500 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-2)]: Done  10 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-2)]: Done  19 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-2)]: Done  28 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-2)]: Done  39 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done  50 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-2)]: Done  63 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-2)]: Done  76 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-2)]: Done  91 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-2)]: Done 106 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-2)]: Done 123 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-2)]: Done 140 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-2)]: Done 159 tasks      | elapsed:  9.9min


In [140]:
best_params = XGsearch.best_params_
print(best_params)

{'selectkbest__k': 100}


In [ ]:
clf2.named_steps['xgbclassifier'].feature_importances_

In [ ]:
pipe2 = make_pipeline(
    PreProcessor,  
    LogisticRegression())

param_grid2 = {
    'logisticregression__C': [0.01, 0.1, 1, 3 10]
    }

# Fit on the train set, with grid search cross-validation
gs2 = GridSearchCV(pipe2, param_grid=param_grid2, cv=3, 
                      scoring='accuracy', 
                      verbose=10, n_jobs=-2)
gs2.fit(X_train, y_train)


print("model score: %.3f" % gs2.score(X_train, y_train))
print("model score: %.3f" % gs2.score(X_val, y_val))

### Trying with Ridge Classifier

In [ ]:
pipe = make_pipeline(
    PreProcessor, 
    SelectKBest(f_classif), 
    RidgeClassifier())

param_grid = {
    'selectkbest__k': range(1, len(X_train.columns)+1), 
    'ridgeclassifier__alpha': [0.001, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
    }

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=5, 
                      scoring='accuracy', 
                      verbose=10, n_jobs=-2)
gs.fit(X_train, y_train)
print("model score: %.3f" % gs.score(X_train, y_train))
print("model score: %.3f" % gs.score(X_val, y_val))

### Deep diving on some of these features.


In [ ]:
print(data.ward.value_counts()[data.funder.value_counts()>777])
print(len(list(data.ward.value_counts()[data.funder.value_counts()<777])))

In [ ]:
print(data.wpt_name.value_counts()[data.wpt_name.value_counts()>100])
print(len(list(data.wpt_name.value_counts()[data.wpt_name.value_counts()<100])))

In [ ]:
data.T.head(40)

In [ ]:
data.waterpoint_type.value_counts()

In [ ]:
df.region_code.nunique()

In [ ]:
from sklearn.feature_selection import RFECV
clf8 = make_pipeline(PreProcessor, RFECV(estimator=LogisticRegression(), step=1, min_features_to_select=1, cv=3, scoring='accuracy', verbose=10, n_jobs=-2))
clf8.fit(X_train, y_train)
print("model score: %.3f" % clf8.score(X_train, y_train))
print("model score: %.3f" % clf8.score(X_val, y_val))

## Pre-process the test data, make predictions, and format submission

In [132]:
clf2 = make_pipeline(PreProcessor, XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.02,  
                      colsample_bytree = 0.3,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=750, 
                      reg_alpha = 0.4,
                      max_depth=17, 
                      gamma=1))

clf2.fit(X, y)
print("model score: %.3f" % clf2.score(X_train, y_train))
print("model score: %.3f" % clf2.score(X_val, y_val))

[23:11:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3482 extra nodes, 922 pruned nodes, max_depth=17
[23:11:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2696 extra nodes, 766 pruned nodes, max_depth=17
[23:11:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3020 extra nodes, 876 pruned nodes, max_depth=17
[23:11:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3248 extra nodes, 622 pruned nodes, max_depth=17
[23:11:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3188 extra nodes, 664 pruned nodes, max_depth=17
[23:11:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2748 extra nodes, 682 pruned nodes, max_depth=17
[23:11:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3206 extra nodes, 678 pruned nodes, max_depth=17
[23:11:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3878 extra nodes, 912 pru

[23:11:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2336 extra nodes, 1198 pruned nodes, max_depth=17
[23:11:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2866 extra nodes, 1554 pruned nodes, max_depth=17
[23:11:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2964 extra nodes, 1418 pruned nodes, max_depth=17
[23:11:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2360 extra nodes, 1424 pruned nodes, max_depth=17
[23:11:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3202 extra nodes, 1774 pruned nodes, max_depth=17
[23:11:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2948 extra nodes, 1476 pruned nodes, max_depth=17
[23:11:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2198 extra nodes, 1938 pruned nodes, max_depth=17
[23:11:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2554 extra nodes, 

[23:11:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1512 extra nodes, 1734 pruned nodes, max_depth=17
[23:11:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1796 extra nodes, 1426 pruned nodes, max_depth=17
[23:11:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1492 extra nodes, 2140 pruned nodes, max_depth=17
[23:11:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1468 extra nodes, 2388 pruned nodes, max_depth=17
[23:11:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2034 extra nodes, 1596 pruned nodes, max_depth=17
[23:11:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1870 extra nodes, 1488 pruned nodes, max_depth=17
[23:11:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1534 extra nodes, 1484 pruned nodes, max_depth=17
[23:11:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1964 extra nodes, 

[23:12:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1642 extra nodes, 1572 pruned nodes, max_depth=17
[23:12:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1032 extra nodes, 2678 pruned nodes, max_depth=17
[23:12:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1704 extra nodes, 2110 pruned nodes, max_depth=17
[23:12:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1860 extra nodes, 1794 pruned nodes, max_depth=17
[23:12:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1138 extra nodes, 2080 pruned nodes, max_depth=17
[23:12:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1538 extra nodes, 1814 pruned nodes, max_depth=17
[23:12:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1128 extra nodes, 2718 pruned nodes, max_depth=17
[23:12:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1132 extra nodes, 

[23:12:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1456 extra nodes, 1854 pruned nodes, max_depth=17
[23:12:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1026 extra nodes, 860 pruned nodes, max_depth=17
[23:12:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1004 extra nodes, 1408 pruned nodes, max_depth=17
[23:12:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 922 extra nodes, 1654 pruned nodes, max_depth=17
[23:12:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 658 extra nodes, 2342 pruned nodes, max_depth=17
[23:12:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 636 extra nodes, 2694 pruned nodes, max_depth=17
[23:12:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 756 extra nodes, 1824 pruned nodes, max_depth=17
[23:12:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1092 extra nodes, 1818 

...

[23:13:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 932 extra nodes, 1780 pruned nodes, max_depth=17
[23:13:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 598 extra nodes, 1410 pruned nodes, max_depth=17
[23:13:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 604 extra nodes, 2096 pruned nodes, max_depth=17
[23:13:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 682 extra nodes, 1136 pruned nodes, max_depth=17
[23:13:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 820 extra nodes, 1756 pruned nodes, max_depth=17
[23:13:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 466 extra nodes, 1022 pruned nodes, max_depth=17
[23:13:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 552 extra nodes, 1910 pruned nodes, max_depth=17
[23:13:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 282 extra nodes, 1860 pru

[23:14:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 594 extra nodes, 1010 pruned nodes, max_depth=17
[23:14:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 338 extra nodes, 2382 pruned nodes, max_depth=15
[23:14:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 538 extra nodes, 1192 pruned nodes, max_depth=17
[23:14:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 734 extra nodes, 1450 pruned nodes, max_depth=17
[23:14:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 684 extra nodes, 1222 pruned nodes, max_depth=17
[23:14:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 692 extra nodes, 1666 pruned nodes, max_depth=17
[23:14:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 422 extra nodes, 784 pruned nodes, max_depth=17
[23:14:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 346 extra nodes, 450 prune

[23:14:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 378 extra nodes, 966 pruned nodes, max_depth=17
[23:14:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 448 extra nodes, 1110 pruned nodes, max_depth=17
[23:14:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 356 extra nodes, 2566 pruned nodes, max_depth=16
[23:14:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 388 extra nodes, 586 pruned nodes, max_depth=17
[23:14:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 624 extra nodes, 1032 pruned nodes, max_depth=17
[23:14:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 574 extra nodes, 1650 pruned nodes, max_depth=17
[23:14:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 268 extra nodes, 1112 pruned nodes, max_depth=17
[23:14:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 1978 prune

[23:14:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 422 extra nodes, 1032 pruned nodes, max_depth=17
[23:14:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 242 extra nodes, 754 pruned nodes, max_depth=17
[23:14:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 338 extra nodes, 1820 pruned nodes, max_depth=17
[23:14:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 318 extra nodes, 1012 pruned nodes, max_depth=17
[23:14:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 324 extra nodes, 2056 pruned nodes, max_depth=17
[23:14:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 446 extra nodes, 1048 pruned nodes, max_depth=17
[23:14:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 454 extra nodes, 1360 pruned nodes, max_depth=17
[23:14:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 494 extra nodes, 1046 prun

[23:15:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 356 extra nodes, 1094 pruned nodes, max_depth=17
[23:15:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 224 extra nodes, 508 pruned nodes, max_depth=17
[23:15:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 450 extra nodes, 1784 pruned nodes, max_depth=17
[23:15:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 1938 pruned nodes, max_depth=17
[23:15:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 510 extra nodes, 1044 pruned nodes, max_depth=17
[23:15:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 232 extra nodes, 2408 pruned nodes, max_depth=17
[23:15:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 538 extra nodes, 1808 pruned nodes, max_depth=17
[23:15:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 232 extra nodes, 2194 prun

In [133]:
print(df_test.shape)
test_data = lat_long(df_test)
test_data['status_group'] = df_test.id
test_data = no_bool(test_data, ['permit', 'public_meeting'])
#test_data = PreProcessor.fit_transform(test_data)
print(test_data.shape)

(14358, 40)
(14358, 42)


In [134]:
#Make predictions using the features from the test data set
predictions = clf2.predict(test_data)

#Display our predictions - they are either 0 or 1 for each training instance 
#depending on whether our algorithm believes the person survived or not.
predictions

# Map our labels back onto the predictions
keys = label_list[0]
values = label_list[1]
status_labels = dict(zip(keys, values))
predictions_mapped = np.vectorize(status_labels.get)(predictions)

In [135]:
#Create a  DataFrame with the passengers ids and our prediction regarding whether they survived or not
submission = pd.DataFrame({'id':df_test['id'],'status_group':predictions_mapped})

# make sure the shapes line up. 
print (df_test.shape)
print (submission.shape)

#Visualize the first 5 rows
submission.head()

(14358, 40)
(14358, 2)


,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [136]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as your notebook
filename = 'submission.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: submission.csv


# MISC Ref Material

In [ ]:
def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:-1]:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

get_column_names_from_ColumnTransformer(PreProcessor)

In [ ]:
# Rewriting my Dummy Regression Baseline one as a function
def baseline(data):
    name = "Dummy Regression Baseline"
    # Split data into train and test
    X_train, X_test, y_train, y_test = split(data)

    # Define an estimator and param_grid
    # WHEN DEFINING YOU CAN GIVE IT A NAME OTHERWISE IT WILL USE THE PIPELINE NAME AUTOGEN NAME (name of the function but lowercase)
    pipe = make_pipeline(
        PreProcesser(), 
        DummyRegressor(strategy='mean'))
    pipe.fit(X_train, y_train)

    scorer = 'MAE'

    ### Get the scores with the MAE Function
    y_pred_train = pipe.predict(X_train)  
    y_pred_test = pipe.predict(X_test)

    train_score = mean_absolute_error(y_train, y_pred_train)
    test_score = mean_absolute_error(y_test, y_pred_test)
    score_variance = test_score - train_score
    cv_score = 0.0000000000000
    best_params = pipe.get_params
    best_estimator = ""
    selected_names = list(X_train.columns)
    unselected_names = []

    return [name, scorer, train_score, test_score, score_variance, cv_score, selected_names, unselected_names, best_params, best_estimator]


# Rewriting my GridSearch CV as a function 
def compare(data, name):
    X_train, X_test, y_train, y_test = split(data)

    pipe = make_pipeline(
        PreProcessor, 
        SelectKBest(f_regression), 
        Ridge())

    param_grid = {
        'selectkbest__k': range(1, len(X_train.columns)+1), 
        'ridge__alpha': [0.1, 1.0, 10.]
    }

    scorer = 'MAE'

    # Fit on the train set, with grid search cross-validation
    gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                      scoring='neg_mean_absolute_error', 
                      verbose=0)
    gs.fit(X_train, y_train)

    train_score = -gs.score(X_train, y_train)
    test_score = -gs.score(X_test, y_test)
    score_variance = test_score - train_score
    cv_score = -gs.best_score_
    best_params = gs.best_params_
    best_estimator = gs.best_estimator_

    # selected features? 
    # 'selectkbest' is the autogenerated name of the SelectKBest() function in the pipeline
    selector = gs.best_estimator_.named_steps['selectkbest']
    all_names = X_train.columns

    # get_support returns a mask of the columns in True / False
    selected_mask = selector.get_support()
    # Passing the boolean list as the column names creates a masked list.  
    selected_names = list(all_names[selected_mask])
    unselected_names = list(all_names[~selected_mask])

    return [name, scorer, train_score, test_score, score_variance, cv_score, selected_names, unselected_names, best_params, best_estimator]

In [ ]:
#pd.DataFrame({'Variable':X.columns,
#              'Importance':clf2.named_steps['xgbclassifier'].feature_importances_}).sort_values('Importance', ascending=False)
#pipe.steps[0][1].get_feature_names()